## **Data Wrangling with Spark SQL**

In [1]:
# importing packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, desc, asc, sum as Fsum
from pyspark.sql.types import IntegerType, StringType

import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# initiating a spark session
spark = SparkSession\
        .builder\
        .appName('Data Wrangling with Spark SQL')\
        .getOrCreate()

23/03/21 05:40:56 WARN Utils: Your hostname, natasha resolves to a loopback address: 127.0.1.1; using 192.168.43.187 instead (on interface wlo1)
23/03/21 05:40:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/21 05:40:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/21 05:41:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# loading data
user_log = spark.read.json('/home/natasha/Documents/Github/Spark-and-Data-Lakes/data/sparkify_log_small.json')
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



### Running Queries

In [5]:
user_log.createOrReplaceTempView("user_log_table")

In [7]:
spark.sql('''
    SELECT *
    FROM user_log_table
    LIMIT 2;
''').show()

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|   Lily Allen|Logged In|Elizabeth|     F|            7|   Chase|195.23873| free|Shreveport-Bossie...|   PUT|NextSong|1512718541284|     5027|      

In [8]:
spark.sql('''
    SELECT DISTINCT(page) AS page
    FROM user_log_table
    ORDER BY page;
''').show()

+----------------+
|            page|
+----------------+
|           About|
|       Downgrade|
|           Error|
|            Help|
|            Home|
|           Login|
|          Logout|
|        NextSong|
|   Save Settings|
|        Settings|
|Submit Downgrade|
|  Submit Upgrade|
|         Upgrade|
+----------------+



In [9]:
spark.sql('''
    SELECT userID, firstname, page, song
    FROM user_log_table
    WHERE userId == '1046';
''').show()

+------+---------+--------+--------------------+
|userID|firstname|    page|                song|
+------+---------+--------+--------------------+
|  1046|  Kenneth|NextSong|Christmas Tears W...|
|  1046|  Kenneth|NextSong|  Be Wary Of A Woman|
|  1046|  Kenneth|NextSong|   Public Enemy No.1|
|  1046|  Kenneth|NextSong|Reign Of The Tyrants|
|  1046|  Kenneth|NextSong|      Father And Son|
|  1046|  Kenneth|NextSong|               No. 5|
|  1046|  Kenneth|NextSong|           Seventeen|
|  1046|  Kenneth|    Home|                null|
|  1046|  Kenneth|NextSong|          War on war|
|  1046|  Kenneth|NextSong|   Killermont Street|
|  1046|  Kenneth|NextSong|        Black & Blue|
|  1046|  Kenneth|  Logout|                null|
|  1046|  Kenneth|    Home|                null|
|  1046|  Kenneth|NextSong|     Heads Will Roll|
|  1046|  Kenneth|NextSong|Bleed It Out [Liv...|
|  1046|  Kenneth|NextSong|              Clocks|
|  1046|  Kenneth|NextSong|           Love Rain|
|  1046|  Kenneth|Ne

In [21]:
spark.udf.register('get_hour', lambda x: int(datetime.datetime.fromtimestamp(x/1000).hour))
spark.sql('''
    SELECT *, get_hour(ts) AS hour
    FROM user_log_table
    LIMIT 1;
''').collect()

23/03/21 06:05:34 WARN SimpleFunctionRegistry: The function get_hour replaced a previously registered function.


[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046', hour='1')]

In [26]:
spark.sql('''
    SELECT get_hour(ts) AS hour, COUNT(*) AS no_plays
    FROM user_log_table
    WHERE page == 'NextSong'
    GROUP BY hour
    ORDER BY cast(hour as int);
''').show()

+----+--------+
|hour|no_plays|
+----+--------+
|   0|     248|
|   1|     369|
|   2|     375|
|   3|     456|
|   4|     454|
|   5|     382|
|   6|     302|
|   7|     352|
|   8|     276|
|   9|     348|
|  10|     358|
|  11|     375|
|  12|     249|
|  13|     216|
|  14|     228|
|  15|     251|
|  16|     339|
|  17|     462|
|  18|     479|
|  19|     484|
+----+--------+
only showing top 20 rows



In [28]:
songs_in_hour = spark.sql('''
    SELECT get_hour(ts) AS hour, COUNT(*) AS no_plays
    FROM user_log_table
    WHERE page == 'NextSong'
    GROUP BY hour
    ORDER BY cast(hour as int);
''')
songs_in_hour = songs_in_hour.toPandas()
songs_in_hour

,hour,no_plays
0,0,248
1,1,369
2,2,375
3,3,456
4,4,454
5,5,382
6,6,302
7,7,352
8,8,276
9,9,348
